## Create a knowledge graph and populate it using the ArcGIS API for Python

In [1]:
# import libraries
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph
from getpass import getpass
from uuid import UUID

The first step to creating a knowledge graph is to create the service in the portal. This can be accomplished with the [`create_service`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.create_service) function in the ArcGIS API for Python.

In [2]:
gis = GIS("<insert portal url>", "<insert username>", getpass())
result = gis.content.create_service(
    name="data_creation",
    capabilities="Query,Editing,Create,Update,Delete",
    service_type="KnowledgeGraph"
)
knowledge_graph = KnowledgeGraph(result.url, gis=gis)

········


We can start to populate the graph by creating a single entity type with the properties we want to populate. This is done using [`named_object_type_adds()`](https://developers.arcgis.com/python/api-reference/arcgis.graph.html) in the arcgis.graph module.

In [3]:
knowledge_graph.named_object_type_adds(
    entity_types=[
        # one dictionary for each type, containing information about that type (optionally including properties to add)
        {
            # at a minimum, a name is required to create a new type. other details can be added as needed.
            "name": "Person",
            # can either add all properties right away or add them later
            "properties": {
                # minimum needed for creating a property type is the name and the role, more can be included as needed
                # fieldType defaults to esriFieldTypeString, so is only really needed for different types
                "name": {
                    "name": "name",
                    "fieldType": "esriFieldTypeString",
                    "role": "esriGraphPropertyRegular"
                },
                "role": {
                    "name": "role",
                    "role": "esriGraphPropertyRegular"
                },
                "shape": {
                    "name": "shape",
                    "fieldType": "esriFieldTypeGeometry",
                    "geometryType": "esriGeometryPoint",
                    "role": "esriGraphPropertyRegular"
                }
            }
        }
    ]
)

{'entityAddResults': [{'name': 'Person'}], 'relationshipAddResults': []}

Now that the entity type has been added to the data model, we can create data in the graph of that type. This is done using `apply_edits()`.

In [4]:
knowledge_graph.apply_edits(adds=[{
            "_objectType": "entity",
            "_typeName": "Person",
            "_properties": {
                "name": "Megan", 
                "role": "Senior Product Engineer",
                "shape": {
                    'x': -89.366878,
                    'y': 43.084730,
                    'spatialReference': {'wkid': 4326},
                    '_objectType': 'geometry'
                }
            }
        },{
            "_objectType": "entity",
            "_typeName": "Person",
            "_properties": {
                "name": "Tim",
                "role": "Director of Contextual Intelligence",
                "shape": {
                    'x': -77.325666,
                    'y': 38.907065,
                    'spatialReference': {'wkid': 4326},
                    '_objectType': 'geometry'
                }
            }
        }
    ]
)

{'editsResult': {'Person': {'addResults': [{'id': UUID('5f7d84af-78ce-41a2-a7b4-d7e4215663b2')},
    {'id': UUID('e95c4a4b-5fd0-461f-b2c1-3938570beca7')}]}},
 'cascadedDeletes': {},
 'relationshipSchemaChanges': {},
 'cascadedProvenanceDeletes': []}

Add a relationship type and additional entity type to the data model using another `named_object_type_adds`.

In [5]:
knowledge_graph.named_object_type_adds(
    entity_types=[
        {
            "name": "Company",
            "properties": {
                "name": {
                    "name": "name",
                    "fieldType": "esriFieldTypeString",
                    "role": "esriGraphPropertyRegular"
                }
            }
        }
    ],
    relationship_types=[
        {
            "name": "WorksAt"
        }
    ]
)

{'entityAddResults': [{'name': 'Company'}],
 'relationshipAddResults': [{'name': 'WorksAt'}]}

Now that we have all of our entity and relationship types, we can add new entities and relationships to the graph. We will start by adding our company entity.

In [6]:
knowledge_graph.apply_edits(adds=[{
            "_objectType": "entity",
            "_typeName": "Company",
            "_properties": {
                "name": "Esri"
            }
        }
])

{'editsResult': {'Company': {'addResults': [{'id': UUID('5d4ca1e6-1b53-49c5-9bc6-14abe9e5e1e5')}]}},
 'cascadedDeletes': {},
 'relationshipSchemaChanges': {},
 'cascadedProvenanceDeletes': []}

To create relationships between the entities, we need to know their unique ids. These can be found through a graph query using `query()`.

In [7]:
# get each entity unique id from a query
megan = knowledge_graph.query("MATCH (p:Person {name: 'Megan'}) RETURN p.globalid")[0][0]
tim = knowledge_graph.query("MATCH (p:Person {name: 'Tim'}) RETURN p.globalid")[0][0]
esri = knowledge_graph.query("MATCH (c:Company {name: 'Esri'}) RETURN c.globalid")[0][0]

# create relationships in the graph
knowledge_graph.apply_edits(adds=[{
            "_objectType": "relationship",
            "_typeName": "WorksAt",
            "_originEntityId": megan,
            "_destinationEntityId": esri
        },
    {
        "_objectType": "relationship",
        "_typeName": "WorksAt",
        "_originEntityId": tim,
        "_destinationEntityId": esri
    }
])

{'editsResult': {'WorksAt': {'addResults': [{'id': UUID('977ac517-dfc7-46f8-bb95-fedb9f849a37')},
    {'id': UUID('a80021be-bed9-4836-906a-5c109f4cfa1b')}]}},
 'cascadedDeletes': {},
 'relationshipSchemaChanges': {'WorksAt': {'new_end_points': [{'origin_entity_type': 'Person',
     'dest_entity_type': 'Company'}]}},
 'cascadedProvenanceDeletes': []}

We did a query to find the entities we were interested in before, so let's take a look at what returning an entity and relationship looks like.

In [8]:
knowledge_graph.query("MATCH (p:Person)-[w:WorksAt]->(c:Company) RETURN p,w,c LIMIT 1")

[[{'_objectType': 'entity',
   '_typeName': 'Person',
   '_id': UUID('5f7d84af-78ce-41a2-a7b4-d7e4215663b2'),
   '_properties': {'role': 'Senior Product Engineer',
    'shape': {'x': -89.366878,
     'y': 43.08473,
     'spatialReference': {'wkid': 4326},
     '_objectType': 'geometry'},
    'name': 'Megan',
    'globalid': UUID('5f7d84af-78ce-41a2-a7b4-d7e4215663b2'),
    'objectid': 1}},
  {'_objectType': 'relationship',
   '_typeName': 'WorksAt',
   '_id': UUID('977ac517-dfc7-46f8-bb95-fedb9f849a37'),
   '_originEntityId': UUID('5f7d84af-78ce-41a2-a7b4-d7e4215663b2'),
   '_destinationEntityId': UUID('5d4ca1e6-1b53-49c5-9bc6-14abe9e5e1e5'),
   '_properties': {'globalid': UUID('977ac517-dfc7-46f8-bb95-fedb9f849a37'),
    'objectid': 1}},
  {'_objectType': 'entity',
   '_typeName': 'Company',
   '_id': UUID('5d4ca1e6-1b53-49c5-9bc6-14abe9e5e1e5'),
   '_properties': {'name': 'Esri',
    'globalid': UUID('5d4ca1e6-1b53-49c5-9bc6-14abe9e5e1e5'),
    'objectid': 1}}]]

openCypher can also be used to get specific information from the entities and relationships.

In [9]:
knowledge_graph.query("MATCH (p:Person {name: 'Megan'}) RETURN p.role")

[['Senior Product Engineer']]

Updates can be made to the entities and relationships at any time using `apply_edits()`.

In [10]:
knowledge_graph.apply_edits(updates= [{
    "_objectType": "entity",
    "_typeName": "Person",
    "_id": megan,
    "_properties": {"role": "Graphiest Product Engineer"}
}])

{'editsResult': {'Person': {'updateResults': [{'id': UUID('5f7d84af-78ce-41a2-a7b4-d7e4215663b2')}]}},
 'cascadedDeletes': {},
 'relationshipSchemaChanges': {},
 'cascadedProvenanceDeletes': []}

Using `query_streaming` we can pass information in as bind parameters to make queries more resuable with different input data.

In [11]:
result = knowledge_graph.query_streaming("MATCH (p:Person {name: $name}) RETURN p.role", bind_param={"name": "Megan"})
list(result)

[['Graphiest Product Engineer']]

In [12]:
knowledge_graph.graph_property_adds(
    type_name="Person", 
    graph_properties=[{
        "name": "employee_id", 
        "fieldType": "esriFieldTypeInteger",
        "role": "esriGraphPropertyRegular"
    }]
)

{'propertyAddResults': [{'name': 'employee_id'}]}

In [13]:
knowledge_graph.apply_edits(updates=[{
        "_objectType": "entity",
        "_typeName": "Person",
        "_id": megan,
        "_properties": {"employee_id": 392472034}
    },{
        "_objectType": "entity",
        "_typeName": "Person",
        "_id": tim,
        "_properties": {"employee_id": 8247302}
    }]
)

{'editsResult': {'Person': {'updateResults': [{'id': UUID('5f7d84af-78ce-41a2-a7b4-d7e4215663b2')},
    {'id': UUID('e95c4a4b-5fd0-461f-b2c1-3938570beca7')}]}},
 'cascadedDeletes': {},
 'relationshipSchemaChanges': {},
 'cascadedProvenanceDeletes': []}

In [14]:
knowledge_graph.apply_edits(deletes=[{
    "_objectType": "entity",
    "_typeName": "Person",
    "_ids": [tim]
}], cascade_delete=True)

{'editsResult': {'Person': {'deleteResults': [{'id': UUID('e95c4a4b-5fd0-461f-b2c1-3938570beca7')}]}},
 'cascadedDeletes': {'WorksAt': [{'id': UUID('a80021be-bed9-4836-906a-5c109f4cfa1b'),
    'originId': UUID('e95c4a4b-5fd0-461f-b2c1-3938570beca7'),
    'destId': UUID('5d4ca1e6-1b53-49c5-9bc6-14abe9e5e1e5')}]},
 'relationshipSchemaChanges': {},
 'cascadedProvenanceDeletes': []}

## Error examples

In [15]:
knowledge_graph.apply_edits(updates=[{
        "_objectType": "entity",
        "_typeName": "Person",
        "_id": megan,
        "_properties": {"Employee_id": 392472034}
    }]
)

{'error': {'error_code': 111215,
  'error_message': "Apply edits operation cannot be performed. The apply edits payload contains 'Employee_id' field in 'Person' type, which is not present in the data model."},
 'editsResult': {},
 'cascadedDeletes': {},
 'relationshipSchemaChanges': {},
 'cascadedProvenanceDeletes': []}

In [16]:
for entity in knowledge_graph.datamodel['entity_types']:
    print("\nEntity type ", entity, " has properties: ")
    for prop in knowledge_graph.datamodel['entity_types'][entity]['properties']:
        print(prop)


Entity type  Company  has properties: 
objectid
globalid
name

Entity type  Document  has properties: 
objectid
globalid
geometry
metadata
fileExtension
contentType
keywords
text
url
title
name

Entity type  Person  has properties: 
employee_id
objectid
globalid
shape
role
name


In [17]:
knowledge_graph.apply_edits(updates=[{
        "_objectType": "entity",
        "_typeName": "Person",
        "_id": megan,
        "_properties": {"employee_id": "392472034"}
    }]
)

{'error': {'error_code': 111035,
  'error_message': "Apply edit contains data that does not match the data type in data model. Data model contains 'employee_id' field - data type 'esriFieldTypeInteger' in 'Person'. Apply edit data is of type 'esriFieldTypeString'."},
 'editsResult': {},
 'cascadedDeletes': {},
 'relationshipSchemaChanges': {},
 'cascadedProvenanceDeletes': []}

In [18]:
knowledge_graph.apply_edits(updates=[{
        "_objectType": "entity",
        "_typeName": "Person",
        "_id": UUID("102bb72d-ffb1-456e-9b0a-7630e177ba9d"),
        "_properties": {"employee_id": 392472034}
    }]
)

{'error': {'error_code': 0, 'error_message': ''},
 'editsResult': {'Person': {'updateResults': [{'id': UUID('102bb72d-ffb1-456e-9b0a-7630e177ba9d'),
     'error': {'error_code': 111113,
      'error_message': "Failed to update the entity or relationship for 'Person'."}}]}},
 'cascadedDeletes': {},
 'relationshipSchemaChanges': {},
 'cascadedProvenanceDeletes': []}